# **Downloads and Imports**

In [ ]:
!pip install --quiet requests openai jsonschema

In [61]:
import json
from openai import OpenAI
from google.colab import userdata
from typing import List, Dict

# **Configurations, Model Instances and Constans**

In [ ]:
CHAT_MODEL = "llama-3.3-70b-versatile"
STRUCTURED_OUTPUT_MODEL = "openai/gpt-oss-20b"

In [ ]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key = userdata.get("GROQ_API_KEY")
)

In [48]:
MESSAGE_HISTORY : List[dict] = []

In [50]:
SUMMARY = ""

In [51]:
CHAT_MODEL_SYSTEM_PROMPT = """

"""

# **Conversation Manager Utilities**

## Function to add messages

In [64]:
summarization_interval = 8
turn_count = 0
def add_message(message):
    global turn_count
    role = message['role']
    MESSAGE_HISTORY.append(message)
    turn_count += 1
    print(f"Turn {turn_count}: Added {message}")

    if turn_count % summarization_interval == 0:
      print(f"Turn count {turn_count}, output: {turn_count % summarization_interval}")
      periodic_summarization()

## functionm to create summary

In [59]:
def generate_summary(messages):
    global SUMMARY
    global turn_count
    global MESSAGE_HISTORY

    if not MESSAGE_HISTORY:
        print("No messages to summarize.")
        return
    conversation_text = "\n".join([f"{message['role']}: {message['content']}" for message in MESSAGE_HISTORY])

    summary_prompt = f"""
        Please provide a concise summary of the following conversation, in the third person point of view. Focus on:
        1. Main topics discussed
        2. Key decisions or outcomes
        3. Important information exchanged
        4. Overall context and flow

        Conversation:
        {conversation_text}

        Summary:"""
    try:
        chat_bot_response = client.chat.completions.create(
          model=CHAT_MODEL,
            messages=[{"role":"user", "content":conversation_text}],
            max_tokens=150,
            temperature=0.3)
        summary = chat_bot_response.choices[0].message.content.strip()
        print(f"✅ Generated summary ({len(summary)} characters)")
        return summary
    except Exception as e:
        print(f"❌ Error generating summary: {e}")
        return "Summary generation failed"

## Function to create periodic summary

In [60]:
def periodic_summarization():
        global MESSAGE_HISTORY
        global SUMMARY

        # Only generate a new summary if there's enough new conversation
        if len(MESSAGE_HISTORY) >= 2:
            current_summary = generate_summary(MESSAGE_HISTORY)

            if SUMMARY:
                # Combine with previous summary
                combined_prompt = f"""
                Previous summary: {SUMMARY}

                New conversation summary: {current_summary}

                Please create a comprehensive summary that combines both:"""

                try:
                    response = client.chat.completions.create(
                        model=CHAT_MODEL,
                        messages=[{"role": "user", "content": combined_prompt}],
                        max_tokens= 300,
                        temperature=0.3
                    )

                    SUMMARY = response.choices[0].message.content.strip()

                except Exception as e:
                    print(f"❌ Error combining summaries: {e}")
                    SUMMARY = current_summary
            else:
                SUMMARY = current_summary

            # Replace history with summary
            MESSAGE_HISTORY = [
                {"role": "system", "content": CHAT_MODEL_SYSTEM_PROMPT},
                {"role": "system", "content": f"Previous conversation summary: {SUMMARY}"}
            ]

            print(f"Conversation summarized and history reset")
            print(f"Current summary length: {len(SUMMARY)} characters")

## Helper Functions

In [78]:
def count_words(text: str) -> int:
    return len(text.split())

def count_characters(messages: List[Dict]) -> int:
    return sum(len(msg["content"]) for msg in messages)

def count_total_words(messages: List[Dict]) -> int:
    return sum(count_words(msg["content"]) for msg in messages)

## Function to truncate conversation by turns

In [77]:
max_turns = 6
def truncate_by_turns(messages: List[Dict]) -> List[Dict]:
  if max_turns and len(messages) > max_turns:
      truncated = messages[-max_turns:]
      print(f"Truncated to last {max_turns} messages")
      return truncated
  return messages

## Truncate by length

In [71]:
max_characters = 300
max_words = 80
def truncate_by_length(messages: List[Dict]) -> List[Dict]:
    if not (max_characters or max_words):
        return messages

    result = []
    current_chars = 0
    current_words = 0

    # Start from the end to keep most recent messages
    for message in reversed(messages):
        msg_chars = len(message["content"])
        msg_words = count_words(message["content"])

        # Check if adding this message would exceed limits
        if (max_characters and current_chars + msg_chars > max_characters) or (max_words and current_words + msg_words > max_words):
            break

        result.insert(0, message)
        current_chars += msg_chars
        current_words += msg_words

    if len(result) < len(messages):
        removed = len(messages) - len(result)
        print(f"📝 Truncated {removed} messages due to length constraints")

    return result

## Get truncated history

In [76]:
def get_truncated_history(trunc_type:str) -> List[Dict]:
        messages = MESSAGE_HISTORY.copy()
        if not trunc_type:
          raise ValueError("Truncation type not provided")

        # Apply truncation strategies
        if trunc_type == "turns":
          return truncate_by_turns(messages)

        return truncate_by_length(messages)

## Get overall, used parameters

In [79]:
def get_statistics() -> Dict:
    """Get detailed statistics about the conversation."""
    stats = {
            "total_turns": turn_count,
            "current_messages": len(MESSAGE_HISTORY),
            "total_characters": count_characters(MESSAGE_HISTORY),
            "total_words": count_total_words(MESSAGE_HISTORY),
            "has_summary": bool(SUMMARY),
            "summary_length": len(SUMMARY) if SUMMARY else 0,
            "summarization_interval": summarization_interval,
            "truncation_limits": {
                "max_turns": max_turns,
                "max_characters": max_characters,
                "max_words": max_words
            }
        }
    return stats

In [80]:
get_statistics()

{'total_turns': 0,
 'current_messages': 8,
 'total_characters': 1676,
 'total_words': 279,
 'has_summary': True,
 'summary_length': 969,
 'summarization_interval': 8,
 'truncation_limits': {'max_turns': 6, 'max_characters': 300, 'max_words': 80}}

# **Test Bot**

## Try Conversation with LLM

In [ ]:
while True:
    user_input = input("User: ")
    add_message({"role": "user", "content": user_input})

    try:
        chat_bot_response = client.chat.completions.create(
            model=CHAT_MODEL,
            messages=MESSAGE_HISTORY
        )
        response_content = chat_bot_response.choices[0].message.content
        add_message({"role": "assistant", "content": response_content})

        print(f"Bot: {response_content}")

    except Exception as e:
        print(f"An error occurred: {e}")

## Test with test message jsons

In [81]:
conversation_samples = [
        ("user", "Hello! I'm looking for advice on starting a small business."),
        ("assistant", "Great! I'd be happy to help. What type of business are you considering?"),
        ("user", "I'm thinking about opening a coffee shop in downtown Seattle."),
        ("assistant", "Excellent choice! Seattle has a great coffee culture. Have you done any market research?"),
        ("user", "Not yet. Where should I start with market research?"),
        ("assistant", "I recommend starting with competitor analysis, foot traffic studies, and customer surveys."),
        ("user", "That sounds comprehensive. What about financing options?"),
        ("assistant", "For coffee shops, you have several options: SBA loans, traditional bank loans, investors, or crowdfunding.")
    ]

In [83]:
# Clear previous history for a clean test
MESSAGE_HISTORY = []
turn_count = 0
SUMMARY = ""

print("--- Testing Truncation ---")

# fill message history with sample conversation
for role, content in conversation_samples:
    add_message({"role": role, "content": content})

print("\n--- Original Message History ---")
for msg in MESSAGE_HISTORY:
    print(f"{msg['role']}: {msg['content']}")
print(f"Total messages: {len(MESSAGE_HISTORY)}")

print("\n--- Truncating by Turns (max_turns=6) ---")
truncated_by_turns = get_truncated_history(trunc_type="turns")
for msg in truncated_by_turns:
    print(f"{msg['role']}: {msg['content']}")
print(f"Truncated messages: {len(truncated_by_turns)}")


print("\n--- Truncating by Length (max_characters=300, max_words=80) ---")
truncated_by_length = get_truncated_history(trunc_type="length")
for msg in truncated_by_length:
    print(f"{msg['role']}: {msg['content']}")
print(f"Truncated messages: {len(truncated_by_length)}")

print("\n--- Truncation Test Complete ---")

--- Testing Truncation ---
Turn 1: Added {'role': 'user', 'content': "Hello! I'm looking for advice on starting a small business."}
Turn 2: Added {'role': 'assistant', 'content': "Great! I'd be happy to help. What type of business are you considering?"}
Turn 3: Added {'role': 'user', 'content': "I'm thinking about opening a coffee shop in downtown Seattle."}
Turn 4: Added {'role': 'assistant', 'content': 'Excellent choice! Seattle has a great coffee culture. Have you done any market research?'}
Turn 5: Added {'role': 'user', 'content': 'Not yet. Where should I start with market research?'}
Turn 6: Added {'role': 'assistant', 'content': 'I recommend starting with competitor analysis, foot traffic studies, and customer surveys.'}
Turn 7: Added {'role': 'user', 'content': 'That sounds comprehensive. What about financing options?'}
Turn 8: Added {'role': 'assistant', 'content': 'For coffee shops, you have several options: SBA loans, traditional bank loans, investors, or crowdfunding.'}
Tur